In [1]:
import duckdb
con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")



In [2]:
df = con.execute("SELECT * FROM player_matches LIMIT 10").fetchdf()
df

,account_id,match_id,hero_id,team,kills,deaths,assists,denies,net_worth
0,1146874782,35509898,14,Team0,1,5,11,5,34990
1,854939296,35509898,10,Team1,8,6,20,24,32167
2,78706118,35509898,35,Team0,1,6,7,7,23335
3,31357831,35509898,31,Team1,8,2,18,3,35845
4,200135833,35509898,50,Team0,2,9,10,2,28047
5,1030720219,35509898,19,Team1,12,1,14,2,38473
6,129093286,35509898,15,Team1,4,7,14,0,32979
7,158752514,35509898,4,Team1,11,2,18,11,45330
8,99844341,35509898,2,Team1,7,5,14,0,37577
9,193143480,35509898,13,Team0,4,5,6,12,35192


In [ ]:
response = con.execute("""SELECT 
    account_id,
    COUNT(DISTINCT match_id) AS match_count
FROM 
    player_matches
GROUP BY 
    account_id
ORDER BY 
    match_count DESC
LIMIT 25;""").fetchdf()
print(response)

In [ ]:
response = con.execute("""SELECT 
    match_count,
    COUNT(*) AS player_count
FROM (
    SELECT 
        account_id,
        COUNT(DISTINCT match_id) AS match_count
    FROM 
        player_matches
    GROUP BY 
        account_id
) t
GROUP BY 
    match_count
ORDER BY 
    match_count;""").fetchdf()
print(response)

In [ ]:
response = con.execute("""
        SELECT 
            pm.account_id,
            COUNT(DISTINCT pm.match_id) AS match_count,
            MAX(m.start_time) AS match_time
        FROM 
            player_matches pm
        JOIN
            matches m ON pm.match_id = m.match_id
        GROUP BY 
            pm.account_id
        HAVING 
            match_count > 10
        ORDER BY
            match_time DESC
        LIMIT 5
        """).fetchdf()
print(response)

In [2]:
response = con.execute("""
        SELECT 
           for match_id in matches
            count(account_id) in player_trends
        """).fetchdf()
print(response)

ParserException: Parser Error: syntax error at or near "match_id"

In [3]:
df = con.execute("""
    SELECT *
    FROM player_trends
                 limit 25
""").fetchdf()
print(df)

    account_id  p_average_kills  p_average_deaths  p_avg_kd  p_total_matches  \
0    297093245             7.77              8.58      0.91              706   
1    127331261             9.07              7.85      1.16             1023   
2   1190115775             8.85              7.08      1.25             1759   
3      7996759             7.03              5.71      1.23              610   
4   1832319597             7.38              5.92      1.25              424   
5    414858274             5.63              5.28      1.07             1021   
6    205525871             7.27              5.66      1.29             1931   
7    313581868             8.13              7.70      1.06             2344   
8    226345055             8.92              6.33      1.41              795   
9   1862514181             7.10              5.37      1.32              224   
10   107120702            11.93              6.89      1.73              714   
11   440103300             8.95         